In [1]:
%pip install -q pyscbwrapper

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyscbwrapper import SCB
import json
import pandas as pd
import random

In [3]:
scb = SCB('sv', 'BE', 'BE0101', 'BE0101A')
scb.info()

[{'id': 'BefolkningNy',
  'type': 't',
  'text': 'Folkmängden efter region, civilstånd, ålder och kön. År 1968 - 2022',
  'updated': '2023-02-22T08:00:00'},
 {'id': 'FolkmangdNov',
  'type': 't',
  'text': 'Folkmängden den 1 november efter region, ålder och kön. År 2002 - 2022',
  'updated': '2022-12-09T08:00:00'},
 {'id': 'FolkmangdDistrikt',
  'type': 't',
  'text': 'Folkmängden per distrikt, landskap, landsdel eller riket efter kön. År 2015 - 2022',
  'updated': '2023-03-22T08:00:00'},
 {'id': 'BefolkManad',
  'type': 't',
  'text': 'Folkmängden per månad efter region, ålder och kön. År 2000M01 - 2023M09',
  'updated': '2023-11-10T08:00:00'},
 {'id': 'BefolkningR1860N',
  'type': 't',
  'text': 'Folkmängden efter ålder och kön . År 1860 - 2022',
  'updated': '2023-02-22T08:00:00'}]

In [4]:
befolkningNy = SCB('en', 'BE', 'BE0101', 'BE0101A', 'BefolkningNy')
befolkningNy.info()
lan = befolkningNy.get_variables()

In [5]:
counties_and_municipals_in_sweden = lan['region']
counties_and_municipals_in_sweden = counties_and_municipals_in_sweden[1:]

swedish_lan = [s for s in counties_and_municipals_in_sweden if "county" in s.lower()]
swedish_kommun = [s for s in counties_and_municipals_in_sweden if "county" not in s.lower()]

swedish_kommun2 = swedish_kommun[0:188] + swedish_kommun[189:]
for i in range(len(swedish_kommun2)):
    if swedish_kommun2[i] == "Munkfors":
        print(i)

for i in range(len(counties_and_municipals_in_sweden)):
    if counties_and_municipals_in_sweden[i] == "Munkfors":
        print(f"munkfors = {i}")

counties_and_municipals_in_sweden = counties_and_municipals_in_sweden[0:201] + counties_and_municipals_in_sweden[202:] #Removing Munkfors

county_dict = {}
current_key = None

for item in counties_and_municipals_in_sweden:
    if "county" in item.lower():
        current_key = item
        county_dict[current_key] = []
    elif current_key is not None:
        county_dict[current_key].append(item)


with open('variable_data/selfmade_dictionaries/county_dict.json', 'w') as json_file:
    json.dump(county_dict, json_file)

munkfors = 201


In [6]:
df = pd.read_csv("variable_data/downloaded_data/forsamlingsnamn-kommun.csv")
kommun_codes = list(df.Kommun.unique())

swedish_kommun2 = swedish_kommun[0:188] + swedish_kommun[189:]
for i in range(len(swedish_kommun2)):
    if swedish_kommun2[i] == "Munkfors":
        print(i)

codes = dict(zip(kommun_codes, swedish_kommun2))


for k, v in codes.items():
    df = df.replace(k, v)

df = df.drop(columns=["Unnamed: 4", "Unnamed: 5", "Unnamed: 6"])
df['Län'] = df['Län'].astype(str).apply(lambda x: '0' + x if len(x) == 1 else x)

df['Församling'] = df['Församling'].astype(str).apply(lambda x: '0' + x if len(x) == 5 else x)
df.head()

# Create an empty dictionary to store the grouped items
kommuners_forsamlingskoder = {}

# Iterate over the DataFrame and group items based on the 'number_column'
for index, row in df.iterrows():
    Kommun = row['Kommun']
    value = row['Församling']

    if Kommun in kommuners_forsamlingskoder:
        # Append to existing key's list
        kommuners_forsamlingskoder[Kommun].append(value)
    else:
        # Create a new key with a list containing the current value
        kommuners_forsamlingskoder[Kommun] = [value]

print(kommuners_forsamlingskoder)

{'Upplands Väsby': ['011401', '011402', '011403'], 'Vallentuna': ['011501', '011507'], 'Österåker': ['011702', '011703'], 'Värmdö': ['012001', '012003', '012006'], 'Järfälla': ['012304'], 'Ekerö': ['012501', '012503', '012504', '012509'], 'Huddinge': ['012601', '012602', '012603', '012604'], 'Botkyrka': ['012701', '012702'], 'Salem': ['012801'], 'Haninge': ['013601', '013605', '013606'], 'Tyresö': ['013801'], 'Upplands-Bro': ['013901', '013903'], 'Nykvarn': ['014001'], 'Täby': ['016001'], 'Danderyd': ['016201'], 'Sollentuna': ['016301'], 'Stockholm': ['018001', '018004', '018005', '018006', '018007', '018009', '018010', '018011', '018013', '018014', '018015', '018017', '018019', '018021', '018023', '018025', '018027', '018029', '018031', '018034', '018036', '018038', '018039', '018040', '018041', '018090', '018092', '018093'], 'Södertälje': ['018101', '018102', '018104', '018106', '018107', '018108', '018110'], 'Nacka': ['018201', '018202', '018203'], 'Sundbyberg': ['018301'], 'Solna':

In [7]:
distrikt = pd.read_csv("variable_data/downloaded_data/distriktkoder2015.csv")
distrikt = distrikt[['Distriktskod', 'Forsamlingskod']]
distrikt.dropna(inplace=True)
forsamlingskoder = list(kommuners_forsamlingskoder.values())
forsamlingskoder_i_distrikt = list(distrikt.Forsamlingskod.unique())
missing_codes = ['018090', '018092', '018093', '038022', '048422', '048423', '058038', '058039', '061704', '068524', '098075', '098079', '108002', '148025', '238004']

for missing_code in missing_codes:
    for k,v in kommuners_forsamlingskoder.items():
        if missing_code in v:
            kommuners_forsamlingskoder[k].remove(missing_code)

with open('variable_data/selfmade_dictionaries/forsamling_dict.json', 'w') as json_file:
    json.dump(kommuners_forsamlingskoder, json_file)

# Create an empty dictionary to store the grouped items
districtcodes_from_Forsamling = {}

# Iterate over the DataFrame and group items based on the 'number_column'
for index, row in distrikt.iterrows():
    number = str(row['Forsamlingskod'])
    value = str(int(row['Distriktskod']))

    if number in districtcodes_from_Forsamling:
        # Append to existing key's list
        districtcodes_from_Forsamling[number].append(value)
    else:
        # Create a new key with a list containing the current value
        districtcodes_from_Forsamling[number] = [value]

with open('variable_data/selfmade_dictionaries/district_codes_from_Forsamling_dict.json', 'w') as json_file:
    json.dump(districtcodes_from_Forsamling, json_file)

In [8]:
tatorter = pd.read_csv("variable_data/downloaded_data/tatorter2020.csv")
print(tatorter.columns)
tatorter = tatorter[['Kommunnamn', 'Beteckning på tätort']]
tatorter.head()
tatorter['Beteckning på tätort'] = tatorter['Beteckning på tätort'].apply(lambda x: str(x).replace('*', ''))
# Create an empty dictionary to store the grouped items
tatorter_in_municipals = {}

# Iterate over the DataFrame and group items based on the 'number_column'
for index, row in tatorter.iterrows():
    key = row['Kommunnamn']
    value = row['Beteckning på tätort']

    if key in tatorter_in_municipals:
        # Append to existing key's list
        tatorter_in_municipals[key].append(value)
    else:
        # Create a new key with a list containing the current value
        tatorter_in_municipals[key] = [value]

print(tatorter_in_municipals)

# with open('variable_data/selfmade_dictionaries/tatorter_in_municipals_dict.json', 'w') as json_file:
#     json.dump(tatorter_in_municipals, json_file)

Index(['6', 'Länskod', 'Länsnamn', 'Kommunkod', 'Kommunnamn', 'Tätortskod',
       'Beteckning på tätort', 'Del av tätort i annan kommun: 1=Ja',
       'Befolkning 2020-12-31',
       'Andel av total befolkning i kommun 2020-12-31', 'Landareal i hektar',
       'Andel av total landareal kommun '],
      dtype='object')
{'Upplands Väsby': ['Löwenströmska lasarettet', 'Båtbyggartorp och Harva', 'Upplands Väsby och Sollentuna'], 'Vallentuna': ['Brottby', 'Ekskogen, Älgeby och Långsjötorp', 'Ekskogen', 'Johannesudd', 'Karby', 'Kårsta', 'Lindholmen', 'Vallentuna'], 'Täby': ['Vallentuna', 'Arninge', 'Stockholm'], 'Österåker': ['Bammarboda', 'Laggarsvik och Linanäs', 'Täljö', 'Rydbo', 'Solberga', 'Nyhagen och Översättra', 'Stava', 'Nolsjö och Mellansjö', 'Svinninge', 'Spånlöt', 'Näs', 'Åkersberga'], 'Värmdö': ['Björnömalmen och Klacknäset', 'Grönskan, Vedhamn och Baldersnäs', 'Brunn', 'Betsede', 'Djurö', 'Fågelvikshöjden', 'Norra Ingaröstrand och Skälsmara', 'Kopparmora', 'Långvik', 'Norra La

In [9]:
scbinternal_fastlopnr = {}
numbers = [str(x) for x in range(10)]

with open('variable_data/selfmade_dictionaries/county_dict.json', 'r') as json_file:
    counties_with_municipals = json.load(json_file)

with open('variable_data/selfmade_dictionaries/tatorter_in_municipals_dict.json', 'r') as json_file:
    tatorter_in_municipals = json.load(json_file)

with open('variable_data/selfmade_dictionaries/forsamling_dict.json', 'r') as json_file:
    forsamlingar_in_municipals = json.load(json_file)

with open('variable_data/selfmade_dictionaries/district_codes_from_Forsamling_dict.json', 'r') as json_file:
    districtcodes_from_Forsamling = json.load(json_file)


def generate_syntethic_FastBet(municipal_name):
    tatorter = tatorter_in_municipals[municipal_name]
    tatort = tatorter[random.randint(0, len(tatorter)-1)] #Chooses randomly which "Tätort" the person lives in of the ones available to that municipal
    number = str(random.randint(1, 400))
    if random.randint(0, 100) > 30: #It is quite likely that there are numbers followed by the first one
        number += ":" + str(random.randint(1, 400))

    FastBet = f"{municipal_name} {tatort} {number}"
    return FastBet

def generate_synthetic_FastLopNr(FastBet):
    if FastBet in scbinternal_fastlopnr: #If there are people in the same new FastBet then they should have the same FastLopNr as well
        return scbinternal_fastlopnr[FastBet]
    else:
        FastLopNr = "" #If the FastBet does not already exist then we should make a new FastLopNr
        taken_values = set(scbinternal_fastlopnr.values())
        
        while True:
            for _ in range(5):
                FastLopNr = FastLopNr + numbers[random.randint(0, 9)]
            if FastLopNr not in taken_values: #Making sure we don't make the same FastLopNr for different FastBet
                scbinternal_fastlopnr[FastBet] = FastLopNr
                return FastLopNr

def generate_synthetic_Distriktskod(Forsamling):
    possible_districts = districtcodes_from_Forsamling[Forsamling]
    return possible_districts[random.randint(0, len(possible_districts)-1)]

In [10]:
def generate_synthetic_geographical_proximity():
    counties = list(counties_with_municipals.keys())
    Lan = counties[random.randint(0, len(counties)-1)] #Chooses a random Lan to begin with
    municipals = counties_with_municipals[Lan]
    Kommun = municipals[random.randint(0, len(municipals)-1)] #Chooses a random Kommun in that county
    forsamlingar = forsamlingar_in_municipals[Kommun]
    Forsamling = forsamlingar[random.randint(0, len(forsamlingar)-1)] #Chooses a random municipal in that county


    Distriktskod = generate_synthetic_Distriktskod(Forsamling) #Chooses a random district_code based on the Forsamling
    FastBet = generate_syntethic_FastBet(Kommun) #Generates a random Fastighetsbeteckning based on municipal and places to live in that municipal
    FastLopNr = generate_synthetic_FastLopNr(FastBet) #Generates a random FastLopNr that is unique to that FastBet or returns the already existing if the FastBet already exists
    geographical_data = {
                        'Lan'           : [Lan],
                        'Kommun'        : [Kommun],
                        'Forsamling'    : [Forsamling],
                        'Distriktskod'  : [Distriktskod],
                        'FastLopNr'     : [FastLopNr],
                        'FastBet'       : [FastBet]
                        }
    data = pd.DataFrame.from_dict(geographical_data)
    return data

In [11]:
for _ in range(100): #10000 generated people with no errors
    data = generate_synthetic_geographical_proximity()
    data.head()

In [15]:
from variable_functions.generate_all_variables import generate_data

pretty_good_family_data = generate_data(10000)

In [16]:
pretty_good_family_data = pretty_good_family_data.reset_index(drop=True)
pretty_good_family_data.to_csv('pretty_good_family_data.csv')